In [ ]:
import findspark
findspark.init()
findspark.find()

'H:\\SPARK'

In [ ]:
from pyspark.sql import SparkSession

# Initialize SparkSession with necessary configurations
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark') \
    .config("spark.driver.memory", "15g") \
    .config("spark.hadoop.home.dir", "H:/HADOOP/") \
    .config("spark.hadoop.conf.dir", "H:/HADOOP/etc/hadoop/") \
    .getOrCreate()

# Get SparkContext from the SparkSession
sc = spark.sparkContext


In [ ]:
spark

### BASE DATASET

In [ ]:
base_df = spark.read.csv("../../Data/Custom_Datasets/conversation_datasets_GPT.csv", header=True, inferSchema=True)
base_df.show(10, truncate=False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Conversation_ID|Attacker_Helper                                                                                                                                                 |Victim                                                                                                                                                                                         |Conversation_Type|
+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------

### PREPROCESSED DATASET

In [ ]:
preprocessed_df = spark.read.csv("../../Data/Preprocessed_Datasets/GPT_dataset_preprocessed.csv", header=True, inferSchema=True)
preprocessed_df.show(10, truncate=False)

+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Convert Conversation Columns into actual Arrays

In [ ]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import ast

# UDF to convert string representation of list to actual list
def str_to_array_of_arrays(s):
    # Convert the string to a list and then wrap it inside another list
    return [ast.literal_eval(s)][0]

str_to_array_of_arrays_udf = udf(str_to_array_of_arrays, ArrayType(ArrayType(StringType())))

df = preprocessed_df.withColumn("Attacker_Helper", str_to_array_of_arrays_udf(preprocessed_df["Attacker_Helper"])).withColumn("Victim", str_to_array_of_arrays_udf(preprocessed_df["Victim"]))

df.printSchema()

root
 |-- Conversation_ID: string (nullable = true)
 |-- Attacker_Helper: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Victim: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Conversation_Type: integer (nullable = true)



### FLATTEN

In [ ]:
from pyspark.sql.functions import flatten

df = df.withColumn("Attacker_Helper", flatten(df["Attacker_Helper"]))
df = df.withColumn("Victim", flatten(df["Victim"]))

df.show(10, truncate=False)


+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### CONVERTING INTO TF VECTORS
#### Each flattened row in the dataframe is converted to a vector. HashingTF uses the hashing trick. A potential drawback is that multiple words might hash to the same feature index, causing collisions. 

In [ ]:
from pyspark.ml.feature import HashingTF

numFeatures = 2^16

# For Attacker_Helper column
hashingTF_ah = HashingTF(inputCol="Attacker_Helper", outputCol="AH_features", numFeatures=numFeatures) # numFeatures specifies how many features (hash buckets) 2^16 num features is provided here, increase if deemed necessary
df = hashingTF_ah.transform(df)

# For Victim column
hashingTF_v = HashingTF(inputCol="Victim", outputCol="V_features", numFeatures=numFeatures)
df = hashingTF_v.transform(df)

In [ ]:
df.show(10, truncate=True)

+---------------+--------------------+--------------------+-----------------+--------------------+--------------------+
|Conversation_ID|     Attacker_Helper|              Victim|Conversation_Type|         AH_features|          V_features|
+---------------+--------------------+--------------------+-----------------+--------------------+--------------------+
|     GT1sURbxgG|[hi, john, ir, ow...|[im, sorri, dont,...|                1|(18,[0,1,2,4,5,6,...|(18,[0,1,4,5,6,7,...|
|     TwaGOeC96w|[hello, xyz, bank...|[victim, name, he...|                0|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|
|     V73ZDCviQL|[hello, sir, call...|[ive, never, issu...|                1|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|
|     wNsWdbpeld|[hello, john, ban...|[sure, pleas, con...|                0|(18,[0,1,3,4,5,6,...|(18,[0,1,2,3,5,6,...|
| x0pSxAQx1K0abm|[hello, ir, call,...|[oh, idea, give, ...|                1|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|
| I8QHkmsq5AdwAq|[hello, xyz, bank...|[j

### TF-IDF
#### IDF (Inverse Document Frequency): IDF is a measure of how important a term is. While CountVectorizer counts how many times a term appears in a document (Term Frequency or TF), the IDF looks at how often a term appears across all documents.

In [ ]:
from pyspark.ml.feature import IDF

# For Attacker_Helper features
idf_ah = IDF(inputCol="AH_features", outputCol="AH_tfidf_features")
df = idf_ah.fit(df).transform(df)

# For Victim features
idf_v = IDF(inputCol="V_features", outputCol="V_tfidf_features")
df = idf_v.fit(df).transform(df)


df.show(10, truncate=True)

+---------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|Conversation_ID|     Attacker_Helper|              Victim|Conversation_Type|         AH_features|          V_features|   AH_tfidf_features|    V_tfidf_features|
+---------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|     GT1sURbxgG|[hi, john, ir, ow...|[im, sorri, dont,...|                1|(18,[0,1,2,4,5,6,...|(18,[0,1,4,5,6,7,...|(18,[0,1,2,4,5,6,...|(18,[0,1,4,5,6,7,...|
|     TwaGOeC96w|[hello, xyz, bank...|[victim, name, he...|                0|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|
|     V73ZDCviQL|[hello, sir, call...|[ive, never, issu...|                1|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|
|     wNsWdbpeld|[hello, joh

#### New dataframe containing only the conversation type and the tfidf features for Attacker_Helper and Victim columns.

In [ ]:
df = df.select("Conversation_ID", "Conversation_Type","AH_tfidf_features", "V_tfidf_features")

df.show(10, truncate=False)

+---------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Conversation_ID|Conversation_Type|AH_tfidf_features                                                                                                                     

### ASSEMBLING THE TWO INPUT VECTORS INTO ONE

In [ ]:
from pyspark.ml.feature import VectorAssembler


# 1. Feature Assembly
assembler = VectorAssembler(
    inputCols=["AH_tfidf_features", "V_tfidf_features"],
    outputCol="combined_features")

df_assembled = assembler.transform(df)

df_assembled.show(10, truncate=True)

+---------------+-----------------+--------------------+--------------------+--------------------+
|Conversation_ID|Conversation_Type|   AH_tfidf_features|    V_tfidf_features|   combined_features|
+---------------+-----------------+--------------------+--------------------+--------------------+
|     GT1sURbxgG|                1|(18,[0,1,2,4,5,6,...|(18,[0,1,4,5,6,7,...|[0.04110167568555...|
|     TwaGOeC96w|                0|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|[0.16440670274220...|
|     V73ZDCviQL|                1|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|[0.16440670274220...|
|     wNsWdbpeld|                0|(18,[0,1,3,4,5,6,...|(18,[0,1,2,3,5,6,...|[0.28771172979886...|
| x0pSxAQx1K0abm|                1|(18,[0,1,2,3,4,5,...|(18,[0,1,2,3,4,5,...|[0.16440670274220...|
| I8QHkmsq5AdwAq|                0|(18,[0,1,3,4,5,6,...|(18,[0,1,2,3,4,5,...|[0.20550837842775...|
| v9vN0BYTWa0e16|                1|(18,[0,2,3,4,5,6,...|(18,[1,5,7,8,10,1...|(36,[0,2,3,4,5,6,...|
| pVeYjXj2

### Splitting the Data

In [ ]:
(train_data, test_data) = df_assembled.randomSplit([0.8, 0.2], seed=42)
train_data.show(truncate=False)

+---------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# LOGISTIC REGRESSION

### Train the Logistic Regression Model

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(featuresCol="combined_features", labelCol="Conversation_Type", predictionCol="Prediction", maxIter=50)

evaluator = MulticlassClassificationEvaluator(labelCol='Conversation_Type', metricName="accuracy", predictionCol="Prediction")

paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [10, 100]) \
    .addGrid(lr.regParam, [0.01, 0.1, 1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()


# This is similar to cross validation but only split the set once
tvs = TrainValidationSplit(
    estimator=lr, 
    estimatorParamMaps=paramGrid, 
    evaluator=evaluator, 
    trainRatio=0.8,
    parallelism=1, 
    seed=42)


tvsModel = tvs.fit(train_data)

print(tvsModel, '\n')
print(tvsModel.explainParams(), '\n')
print('\nEvaluating with metric -> {}'.format(evaluator.getMetricName()))
print('Evaluators rating for the test set -> {}'.format(evaluator.evaluate(tvsModel.transform(test_data))))


TrainValidationSplitModel_30eb8b6b8b4e 

estimator: estimator to be cross-validated (current: LogisticRegression_c6a783543a4d)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LogisticRegression_c6a783543a4d', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_c6a783543a4d', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_c6a783543a4d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, {Param(parent='LogisticRegression_c6a783543a4d', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_c6a783543a4d', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_c6a783543a4d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an 

In [ ]:
print('Train Validation Split Model validationMetrics -> {}'.format(tvsModel.validationMetrics))
print('Best Model -> {}'.format(tvsModel.bestModel))

Train Validation Split Model validationMetrics -> [0.8636363636363636, 0.8636363636363636, 0.9090909090909091, 0.9090909090909091, 0.8181818181818182, 0.8636363636363636, 0.9090909090909091, 0.7272727272727273, 0.7272727272727273, 0.8636363636363636, 0.8181818181818182, 0.9090909090909091, 0.9090909090909091, 0.8181818181818182, 0.8636363636363636, 0.9090909090909091, 0.7272727272727273, 0.7272727272727273]
Best Model -> LogisticRegressionModel: uid=LogisticRegression_c6a783543a4d, numClasses=2, numFeatures=36


### Testing

In [ ]:
predictions = tvsModel.bestModel.transform(test_data)
predictions.select("Conversation_ID","Conversation_Type", "Prediction", "Probability").where(predictions.Prediction != predictions.Conversation_Type).show(10, truncate=False)

+---------------+-----------------+----------+----------------------------------------+
|Conversation_ID|Conversation_Type|Prediction|Probability                             |
+---------------+-----------------+----------+----------------------------------------+
|ALtIzDsxTHTxIe |0                |1.0       |[0.1801813651939915,0.8198186348060085] |
|GIjmhRS9Dy8xg1 |0                |1.0       |[0.1804922313379496,0.8195077686620504] |
|OHJFQTvA2eXtsV |1                |0.0       |[0.7559038415235954,0.24409615847640465]|
|j2Q9T9FmncM5Re |0                |1.0       |[0.2553681781145947,0.7446318218854053] |
+---------------+-----------------+----------+----------------------------------------+



In [ ]:
model_path = "../Models/Trained_Models/LogisticRegression"
print(type(model_path))

<class 'str'>


In [ ]:
tvsModel.write().overwrite().save(model_path)